In [1]:
import sys
sys.path.append("/usr/lib/python2.7/dist-packages")

In [2]:
import pandas as pd
import numpy as np
import re
import sys
import time
import pickle
import os
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan, bulk
from elasticsearch import ElasticsearchException


from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#from normalization import normalize_corpus,review_to_sentences
from utils import build_feature_matrix
from ast import literal_eval

In [3]:
start_script = time.time()
print ''
print "#######################################################################################"
print "Execution Started............"

def timeit(func):
    """
    Simple timing decorator
    """
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        delta  = time.time() - start
        return result, delta
    return wrapper


def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg



#######################################################################################
Execution Started............


In [4]:
from elasticsearch_dsl import DocType, String, Date, Integer
from elasticsearch_dsl.connections import connections

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
import requests

index = 'ccthinclient'
doc_type = 'thin_client'

elasticsearch_host = '35.167.139.79:9200'
elastic_url = 'http://35.167.139.79:9200/_bulk?pretty=true'
body = {
    "query": {
                    "exists": {
                        "field": "pathforsearch"
                    }
             }
    }

body = {
            "query": {
                "match":{
                        "taggedbyadmin":"false"
                }
            }
    }
body = { 
    "query" : {
        "bool":{
            "filter":{
                "match" : {"taggedbyadmin.keyword":""}
            }
        }
    }
}
model_file_name = "cybercentric_model.pkl"

top_n_tags = 3

single_time_execute_no_documents = 100

threshold_cosine = 0.1

#elasticsearch_host = '52.24.198.221:8080'
#elastic_url = 'http://52.24.198.221:8080/_bulk?pretty=true'

# Define a default Elasticsearch client
client = connections.create_connection(hosts=[elasticsearch_host])

In [5]:
try:
    if not os.path.exists(model_file_name):
        print("Pleae run model before classifying documents")
    else :
        with open(model_file_name, 'rb') as f:
            model = pickle.load(f)
            vectorizer = model.named_steps['vectorizer']
            labels = model.labels_
            tfidf_features = model.features_
            model_build_time = model.timestamp_
except IOError as e:
    print (e.message)

In [6]:
def get_no_of_docs_from_elasticsearch(field):
    body = {
            "query": {
                "exists": {
                    "field": field
                }
            }
        }
    
    s = Search.from_dict(body)
    s = s.index(index)
    s = s.doc_type(doc_type)
    s.using(client)
    return s.count()

test_no = get_no_of_docs_from_elasticsearch('pathforsearch')
print "Total test documents : ", test_no

Total test documents :  58806


In [18]:

from collections import Counter
from itertools import groupby
from itertools import chain


class TagScore:
    def __init__(self, tag, sim_score):
        self.tag = tag
        self.sim_score = sim_score


def get_top_tag(top5):
    toptag_dir = {}
    if len(top5) > 0:
        for row in top5:
            if isinstance(row, TagScore):
                for tag in row.tag:
                    if(tag in toptag_dir and 
                       toptag_dir[str(tag)] > row.sim_score):
                        pass
                    else :
                        toptag_dir[str(tag)] = row.sim_score
    return toptag_dir


def compute_cosine_similarity(doc_features, corpus_features,
                              top_n=3):
    # get document vectors
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    # compute similarities
    similarity = np.dot(doc_features, 
                        corpus_features.T)
    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3))
                            for index in top_docs]
    return top_docs_with_score


def elasticsearch_bulk_update(tags_scores) :
    start_update  = time.time()
    try:
        print('Updating elasticsearch data with Cosine Distance.......')
        response = requests.post(elastic_url, data=''.join(tags_scores))
        print('elasticsearch data updated in ', time.time() - start_update, 'sec')
    except Exception as e:
        print("Error in bulk update to server %s : "%elastic_url, e.message) 
        response = None
    return response

def run_classification(raw_text):
    texts = []
    ids = []
    for row in raw_text:
        texts.append(row['preprocessed_text'])
        ids.append(row['file_id'])
        
    query_docs_tfidf = vectorizer.transform(texts)
    print 'Document Similarity Analysis using Cosine Similarity'
    print('index', '||', 'tagscore', '||', 'top_tag' , '||', 'actual_tag')
    start_cosine = time.time()
    tag_pred_cos = []
    json_update = []
    for index, doc_id in enumerate(ids):

        doc_tfidf = query_docs_tfidf[index]
        top_similar_docs = compute_cosine_similarity(doc_tfidf,
                                                 tfidf_features,
                                                 top_n=top_n_tags)
        top5 = []
        sim_scores = []
        for doc_index, sim_score in top_similar_docs:
            if sim_score > threshold_cosine :
                top5.append(TagScore(labels[doc_index], sim_score))


        tagsscore_dir = get_top_tag(top5)

        tag_pred_cos.append(tagsscore_dir)
        key = ids[index]
        tags = str(tagsscore_dir.keys()).replace('\'', '"')
        tagscore = str(tagsscore_dir).replace('\'', '"')
        predict_time = str(time.time())
        print(index, '|', tagscore, '|', tags, '|')
        json_update.append('''
        { \"update\": { \"_index\": \"ccthinclient\", \"_type\": \"thin_client\", \"_id\": \"%s\", \"_retry_on_conflict\" : 1} }
        { \"script\":{  \"inline\": \"ctx._source.tagscore=params.tagscore;ctx._source.tags=params.tags;ctx._source.predict_time=params.predict_time\", \"params\":{ \"tagscore\":%s, \"tags\":%s, \"predict_time\":%s }   } }'''%(key, tagscore, tags, predict_time))

    json_update.append('''\n''')    
    
    print 'Time taken to predict data cosine : ', time.time() - start_cosine
    return json_update

In [19]:
def read_elasticsearch_data_test(from_i, to_i, field):
    body = {
                "from": from_i,
                "size": to_i,
                "query": {
                    "exists": {
                        "field": field
                    }
                }
            }
    s = Search.from_dict(body)
    s = s.index(index)
    s = s.doc_type(doc_type)
    s.using(client)
    return s.execute()

start_test_read  = time.time()

print 'Reading test documents.........'
raw_text =[]

counter = 0
while test_no > counter * single_time_execute_no_documents:
    raw_text = read_elasticsearch_data_test(counter * single_time_execute_no_documents, single_time_execute_no_documents, "pathforsearch")
    if len(raw_text) > 0:
        json_update = run_classification(raw_text)
        response = elasticsearch_bulk_update(json_update)
        if response is None :
            print("Error in uploading to elasticsearch")
    counter = counter + 1
    

print 'Time taken to read',  test_no ,'train document is : ', time.time() - start_test_read

Reading test documents.........
Document Similarity Analysis using Cosine Similarity
('index', '||', 'tagscore', '||', 'top_tag', '||', 'actual_tag')
(0, '|', '{"mobile": 0.627}', '|', '["mobile"]', '|')
(1, '|', '{"mobile": 0.104}', '|', '["mobile"]', '|')
(2, '|', '{"mobile": 0.134}', '|', '["mobile"]', '|')
(3, '|', '{}', '|', '[]', '|')
(4, '|', '{}', '|', '[]', '|')
(5, '|', '{}', '|', '[]', '|')
(6, '|', '{"mobile": 0.36}', '|', '["mobile"]', '|')
(7, '|', '{"mobile": 0.134}', '|', '["mobile"]', '|')
(8, '|', '{"mobile": 1.0}', '|', '["mobile"]', '|')
(9, '|', '{}', '|', '[]', '|')
(10, '|', '{}', '|', '[]', '|')
(11, '|', '{}', '|', '[]', '|')
(12, '|', '{"mobile": 0.125}', '|', '["mobile"]', '|')
(13, '|', '{}', '|', '[]', '|')
(14, '|', '{"mobile": 0.125}', '|', '["mobile"]', '|')
(15, '|', '{}', '|', '[]', '|')
(16, '|', '{"mobile": 0.128}', '|', '["mobile"]', '|')
(17, '|', '{"mobile": 0.108}', '|', '["mobile"]', '|')
(18, '|', '{"mobile": 0.122}', '|', '["mobile"]', '|')
(1

('elasticsearch data updated in ', 2.6352479457855225, 'sec')
Document Similarity Analysis using Cosine Similarity
('index', '||', 'tagscore', '||', 'top_tag', '||', 'actual_tag')
(0, '|', '{"mobile": 0.145}', '|', '["mobile"]', '|')
(1, '|', '{"mobile": 0.103}', '|', '["mobile"]', '|')
(2, '|', '{}', '|', '[]', '|')
(3, '|', '{}', '|', '[]', '|')
(4, '|', '{}', '|', '[]', '|')
(5, '|', '{}', '|', '[]', '|')
(6, '|', '{"mobile": 0.136}', '|', '["mobile"]', '|')
(7, '|', '{}', '|', '[]', '|')
(8, '|', '{"mobile": 1.0}', '|', '["mobile"]', '|')
(9, '|', '{"mobile": 1.0}', '|', '["mobile"]', '|')
(10, '|', '{"mobile": 0.136}', '|', '["mobile"]', '|')
(11, '|', '{}', '|', '[]', '|')
(12, '|', '{}', '|', '[]', '|')
(13, '|', '{"mobile": 0.128}', '|', '["mobile"]', '|')
(14, '|', '{}', '|', '[]', '|')
(15, '|', '{}', '|', '[]', '|')
(16, '|', '{}', '|', '[]', '|')
(17, '|', '{"mobile": 0.142}', '|', '["mobile"]', '|')
(18, '|', '{"mobile": 0.204}', '|', '["mobile"]', '|')
(19, '|', '{"mobile